# 채팅 모델 미세 조정을 위한 데이터 준비 및 분석

이 노트북은 채팅 모델 미세 조정에 사용되는 채팅 데이터 세트를 전처리하고 분석하는 도구로 사용됩니다.
형식 오류를 확인하고, 기본 통계를 제공하며, 미세 조정 비용을 위한 토큰 수를 추정합니다.
여기에 표시된 방법은 gpt-3.5-turbo의 [현재 미세 조정 방법](https://platform.openai.com/docs/guides/fine-tuning)에 해당합니다.
babbage-002 및 davinci-002와 같은 모델의 경우 [레거시 미세 조정](https://platform.openai.com/docs/guides/legacy-fine-tuning)을 참조하세요.

In [1]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

데이터 로딩 중 ##

먼저 [예제 JSONL 파일](https://github.com/openai/openai-cookbook/blob/main/examples/data/toy_chat_fine_tuning.jsonl)에서 채팅 데이터 세트를 로드합니다.

In [2]:
data_path = "data/toy_chat_fine_tuning.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 5
First example:
{'role': 'system', 'content': 'You are a happy assistant that puts a positive spin on everything.'}
{'role': 'user', 'content': 'I fell off my bike today.'}
{'role': 'assistant', 'content': "It's great that you're getting exercise outdoors!"}


## 형식 유효성 검사

다양한 오류 검사를 수행하여 데이터 세트의 각 대화가 미세 조정 API에서 예상하는 형식을 준수하는지 검증할 수 있습니다. 오류는 그 성격에 따라 분류되어 디버깅이 더 쉬워집니다.

1. **데이터 유형 검사**: 데이터 세트의 각 항목이 사전(`딕트`)인지 여부를 확인합니다. 오류 유형: `데이터_유형`.
2. **메시지 목록 존재 여부**: 각 항목에 `메시지` 목록이 있는지 확인합니다. 오류 유형: `missing_messages_list`.
3. **메시지 키 확인**: 메시지` 목록의 각 메시지에 `역할` 및 `내용` 키가 포함되어 있는지 확인합니다. 오류 유형: `메시지_미싱_키`.
4. **메시지에서 인식할 수 없는 키**: 메시지에 `역할`, `내용`, `이름` 이외의 키가 있는 경우 로그를 기록합니다. 오류 유형: 메시지_미인식_키`.
5. **역할 유효성 검사**: '역할'이 '시스템', '사용자' 또는 '어시스턴트' 중 하나인지 확인합니다. 오류 유형: '인식할 수 없는 역할'.
6. **콘텐츠 유효성 검사**: 콘텐츠`에 텍스트 데이터가 있고 문자열인지 확인합니다. 오류 유형: `missing_content`.
7. **어시스턴트 메시지 존재 여부**: 각 대화에 어시스턴트가 보낸 메시지가 하나 이상 있는지 확인합니다. 오류 유형: `EXAMPLE_MISSING_ASSISTANT_MESSAGE`.

아래 코드는 이러한 검사를 수행하며, 발견된 각 오류 유형에 대한 개수를 출력합니다. 이는 디버깅하고 데이터 세트가 다음 단계에 사용할 준비가 되었는지 확인하는 데 유용합니다.


In [3]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


토큰 카운팅 유틸리티 ##

노트북의 나머지 부분에서 사용할 몇 가지 유용한 유틸리티를 정의해 보겠습니다.

In [4]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

데이터 경고 및 토큰 수 ##

몇 가지 가벼운 분석을 통해 누락된 메시지와 같은 데이터 세트의 잠재적인 문제를 식별하고 메시지 및 토큰 수에 대한 통계적 인사이트를 제공할 수 있습니다.

1. **누락된 시스템/사용자 메시지**: '시스템' 또는 '사용자' 메시지가 누락된 대화의 수를 계산합니다. 이러한 메시지는 어시스턴트의 행동을 정의하고 대화를 시작하는 데 매우 중요합니다.
2. **예시당 메시지 수**: 각 대화에 포함된 메시지 수의 분포를 요약하여 대화의 복잡성에 대한 통찰력을 제공합니다.
3. **예시당 총 토큰 수**: 각 대화의 총 토큰 수 분포를 계산하고 요약합니다. 미세 조정 비용을 이해하는 데 중요합니다.
4. **어시스턴트 메시지의 토큰 수**: 대화당 어시스턴트 메시지의 토큰 수를 계산하고 이 분포를 요약합니다. 어시스턴트의 장황함을 이해하는 데 유용합니다.
5. **토큰 한도 경고**: 최대 토큰 한도(4096개)를 초과하는 예시가 있는지 확인합니다. 이러한 예시는 미세 조정 중에 잘려서 데이터 손실이 발생할 수 있습니다.


In [5]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 1
Num examples missing user message: 1

#### Distribution of num_messages_per_example:
min / max: 2, 9
mean / median: 3.8, 3.0
p5 / p95: 2.0, 6.6000000000000005

#### Distribution of num_total_tokens_per_example:
min / max: 26, 8032
mean / median: 1648.4, 45.0
p5 / p95: 26.8, 4863.6

#### Distribution of num_assistant_tokens_per_example:
min / max: 4, 8000
mean / median: 1610.2, 10.0
p5 / p95: 6.0, 4811.200000000001

1 examples may be over the 4096 token limit, they will be truncated during fine-tuning


비용 견적 ##

이 마지막 섹션에서는 미세 조정에 사용될 총 토큰 수를 추정하여 대략적인 비용을 추정할 수 있습니다. 미세 조정 작업의 기간도 토큰 수에 따라 증가한다는 점에 유의할 필요가 있습니다.

In [6]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~4306 tokens that will be charged for during training
By default, you'll train for 20 epochs on this dataset
By default, you'll be charged for ~86120 tokens


총 비용을 추정하려면 https://openai.com/pricing 을 참조하세요.